# How I Built a Medical RAG Chatbot Using BioMistral|Langchain | FREE Colab|ALL OPENSOURCE #ai

https://www.youtube.com/watch?v=E53hc-jcUeE

## Import Libraries

In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain_community.llms import LlamaCpp

from langchain.chains import RetrievalQA, LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

from langchain_community.llms import Ollama 


import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown

## Functions

In [2]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Import Document

In [3]:
loader = PyPDFDirectoryLoader("pdfs/")
docs = loader.load()

## Text Splitting - Chunking

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

In [ ]:
print(f'Número de chunks: {len(chunks)}')
print(f'Exemplo de chunk: {chunks[0]}')

747

## Embeddings

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

## Vector Store - ChromaDB

In [8]:
vectorstore = Chroma.from_documents(chunks, embeddings)
vectorstore

In [9]:
query = "Why should I care about my heart health?"
search = vectorstore.similarity_search(query)
to_markdown(search[0].page_content)

> Heart Disease: Why Should You Care? . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1
> What You Need To Know About Heart Disease . . . . . . . . . . . . . . . . . . . . . . 3

## Retriever

In [10]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)

In [11]:
retriever.get_relevant_documents(query)

[Document(page_content='Heart Disease: Why Should You Care? . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\nWhat You Need To Know About Heart Disease . . . . . . . . . . . . . . . . . . . . . . 3', metadata={'page': 3, 'source': 'pdfs\\healthyheart.pdf'}),
 Document(page_content='It may be tempting to believe that doing just one healthy thing willtake care of your heart disease risk. For example, you may hopethat if you walk or swim regularly, you can still eat a lot of fattyfoods and stay fairly healthy. Not true. To protect your heart, it isvital to make changes that', metadata={'page': 8, 'source': 'pdfs\\healthyheart.pdf'}),
 Document(page_content='2\nThese facts may seem frightening, but they need not be. The good\nnews is that you have a lot of power to protect and improve yourheart health. This guidebook will help you find out your own riskof heart disease and take steps to prevent it.', metadata={'page': 6, 'source': 'pdfs\\healthyheart.pdf'}),
 Document(page_content

## Large Language Model - Open Source - Gemma:2b

In [12]:
llm = Ollama(model="gemma:2b")

## RAG Chain

In [14]:
template = """
<|context|>
You are an AI Health assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query}
</s>
 <|assistant|>
"""

In [15]:
prompt = ChatPromptTemplate.from_template(template)

In [16]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
response = rag_chain.invoke("Why should I care about my heart health?")

In [18]:
to_markdown(response)

> Your heart is a vital organ that plays a crucial role in maintaining your overall health and well-being. Some of the reasons why it's important to prioritize your heart health include:
> 
> * **Reduced risk of cardiovascular diseases:** Heart disease is the leading cause of death worldwide, and taking steps to maintain a healthy heart can significantly reduce your risk of developing this condition.
> * **Improved quality of life:** A healthy heart can help you enjoy a higher quality of life by reducing symptoms such as shortness of breath, fatigue, and pain.
> * **Increased lifespan:** Studies have shown that people with heart health conditions tend to live longer and healthier lives.
> * **Reduced risk of stroke:** High blood pressure, cholesterol levels, and other risk factors associated with heart disease can increase your chances of stroke.
> * **Improved cognitive function:** Research suggests a possible link between heart health and cognitive function, with a healthy heart associated with improved memory, attention, and concentration.
> * **Protection against chronic diseases:** Heart disease can increase your risk of developing other chronic diseases such as diabetes, obesity, and cancer.
> * **Peace of mind:** Knowing you have taken steps to prioritize your heart health can provide you with peace of mind and reduce your stress levels.